In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
from utils import generate_table, Cell, to_latex, merge_cols

In [3]:
data = pd.read_csv("../../results/cmnist/3_digits/first_decent_results.csv")

In [4]:
data["Group"].value_counts()

.same_as_for2.zs2.even_longer.3dig_4miss    30
Name: Group, dtype: int64

In [5]:
data = data.replace({"Group": {
    ".same_as_for2.zs2.even_longer.3dig_4miss": r"\texttt{ZSF}",
}}, inplace=False)

In [6]:
data.columns

Index(['Name', 'misc.seed', 'Runtime', 'Group', 'Hostname', 'State',
       'bias.log_dataset', 'fdm.balanced_context', 'fdm.lr',
       'misc.data_split_seed', 'misc.exp_group', 'misc.log_method',
       'Accuracy (pytorch_classifier)', 'Accuracy Discriminator (zy)',
       'Accuracy Predictor s', 'Accuracy Predictor y',
       'Accuracy_colour_0.0 (pytorch_classifier)',
       'Accuracy_colour_0.0-colour_1.0 (pytorch_classifier)',
       'Accuracy_colour_0.0-colour_2.0 (pytorch_classifier)',
       'Accuracy_colour_0.0÷colour_1.0 (pytorch_classifier)',
       'Accuracy_colour_0.0÷colour_2.0 (pytorch_classifier)',
       'Accuracy_colour_1.0 (pytorch_classifier)',
       'Accuracy_colour_1.0-colour_2.0 (pytorch_classifier)',
       'Accuracy_colour_1.0÷colour_2.0 (pytorch_classifier)',
       'Accuracy_colour_2.0 (pytorch_classifier)', 'ELBO', 'Learning rate',
       'Loss Adversarial', 'Loss Generator', 'Loss Predictor s',
       'Loss Predictor y', 'Loss reconstruction', 'Prior Loss

In [45]:
def compute_ratio_means(df, metric: str, suffix: str = " (pytorch_classifier)"):
    ratios = (
        df[f"{metric}_colour_0.0÷colour_1.0" + suffix],
        df[f"{metric}_colour_0.0÷colour_2.0" + suffix],
        df[f"{metric}_colour_1.0÷colour_2.0" + suffix],
    )
    mean = 0
    for ratio in ratios:
        assert (ratio <= 1.0).all(), str((ratio < 1.0))
        mean += ratio
    df[f"{metric} ratio mean"] = mean / len(ratios)

In [78]:
def compute_ratio_min(df, metric: str, suffix: str = " (pytorch_classifier)"):
    ratios = (
        df[f"{metric}_colour_0.0÷colour_1.0" + suffix],
        df[f"{metric}_colour_0.0÷colour_2.0" + suffix],
        df[f"{metric}_colour_1.0÷colour_2.0" + suffix],
    )
    min_ = pd.Series(1, ratios[0].index)
    for ratio in ratios:
        min_ = min_.where(min_ < ratio, ratio)
    df[f"{metric} ratio min"] = min_

In [79]:
def compute_diff_max(df, metric: str, suffix: str = " (pytorch_classifier)"):
    diffs = (
        df[f"{metric}_colour_0.0-colour_1.0" + suffix],
        df[f"{metric}_colour_0.0-colour_2.0" + suffix],
        df[f"{metric}_colour_1.0-colour_2.0" + suffix],
    )
    max_ = pd.Series(0, diffs[0].index)
    for diff in diffs:
        max_ = max_.where(max_ > diff, diff)
    df[f"{metric} diff max"] = max_

In [80]:
compute_ratio_means(data, "prob_pos")
compute_ratio_means(data, "TPR")
compute_ratio_means(data, "TNR")
compute_ratio_min(data, "prob_pos")
compute_ratio_min(data, "TPR")
compute_ratio_min(data, "TNR")
compute_diff_max(data, "prob_pos")
compute_diff_max(data, "TPR")
compute_diff_max(data, "TNR")

In [81]:
cols = {
#     'Clust/Context Accuracy': "Clust Acc",
#     'Clust/Context NMI': "Clust NMI",
#     'Clust/Context ARI': "Clust ARI",
    "Accuracy (pytorch_classifier)": "Accuracy",
#     'prob_pos_colour_0.0÷colour_1.0 (pytorch_classifier)': "AR ratio",
    "Renyi preds and s (pytorch_classifier)": "Renyi corr",
#     "TPR_colour_0.0÷colour_1.0 (pytorch_classifier)": "TPR ratio",
#     'TNR_colour_0.0÷colour_1.0 (pytorch_classifier)': "TNR ratio",
    "prob_pos ratio mean": "AR ratio mean",
    "TPR ratio mean": "TPR ratio mean",
    "TNR ratio mean": "TNR ratio mean",
    "prob_pos ratio min": "AR ratio min",
    "prob_pos diff max": "AR diff max",
}

In [82]:
groupby = "Group"
res = (data[[groupby] + list(cols)]
        .groupby([groupby])
        .agg(Cell(round_to=3)).rename(columns=cols, inplace=False)
        .reset_index(level=[groupby], inplace=False)
        .rename(columns={groupby: "Method"}, inplace=False)
       )
res

,Method,Accuracy,Renyi corr,AR ratio mean,TPR ratio mean,TNR ratio mean,AR ratio min,AR diff max
0,\texttt{ZSF},0.882 $\pm$ 0.076,0.264 $\pm$ 0.169,0.725 $\pm$ 0.15,0.911 $\pm$ 0.118,0.798 $\pm$ 0.201,0.604 $\pm$ 0.213,0.236 $\pm$ 0.189


In [83]:
# baseline files
baseline_files = {
    r"\texttt{Kamiran \& Calders (2012) CNN}": "cmnist_cnn_baseline_color_60epochs.csv",
    r"\texttt{Kamiran \& Calders (2012) CNN} (more stable)": "bs512_lr1e-3_wd1e-4_cnn_baseline_20epochs.csv",
    r"\texttt{FWD \cite{HasSriNamLia18}}": "cmnist_dro_baseline_color_60epochs.csv",
}

In [84]:
def collate(file_dict, dir_):
    df_all = pd.DataFrame()
    for log_method, filename in file_dict.items():
        df = pd.read_csv(Path("../../results/cmnist/3_digits") / dir_ / filename)
#         df.insert(0, "exp_group", exp_group)
        df.insert(0, "log_method", log_method)
        if log_method == r"\texttt{FWD \cite{HasSriNamLia18}}":
            df["log_method"] = log_method + " (" + df["eta"].astype(str) + ")"
        df_all = pd.concat([df_all, df], axis="index", ignore_index=True, sort=False)
    return df_all

In [85]:
baselines = collate(baseline_files, ".")

In [86]:
compute_ratio_means(baselines, "prob_pos", "")
compute_ratio_means(baselines, "TPR", "")
compute_ratio_means(baselines, "TNR", "")
compute_ratio_min(baselines, "prob_pos", "")
compute_ratio_min(baselines, "TPR", "")
compute_ratio_min(baselines, "TNR", "")
compute_diff_max(baselines, "prob_pos", "")
compute_diff_max(baselines, "TPR", "")
compute_diff_max(baselines, "TNR", "")

In [87]:
baselines.columns

Index(['log_method', 'seed', 'data', 'method', 'wandb_url', 'Accuracy', 'TPR',
       'TNR', 'Renyi preds and s', 'Accuracy_colour_2.0',
       'Accuracy_colour_1.0', 'Accuracy_colour_0.0',
       'Accuracy_colour_0.0-colour_1.0', 'Accuracy_colour_0.0-colour_2.0',
       'Accuracy_colour_1.0-colour_2.0', 'Accuracy_colour_0.0÷colour_1.0',
       'Accuracy_colour_0.0÷colour_2.0', 'Accuracy_colour_1.0÷colour_2.0',
       'prob_pos_colour_2.0', 'prob_pos_colour_1.0', 'prob_pos_colour_0.0',
       'prob_pos_colour_0.0-colour_1.0', 'prob_pos_colour_0.0-colour_2.0',
       'prob_pos_colour_1.0-colour_2.0', 'prob_pos_colour_0.0÷colour_1.0',
       'prob_pos_colour_0.0÷colour_2.0', 'prob_pos_colour_1.0÷colour_2.0',
       'TPR_colour_2.0', 'TPR_colour_1.0', 'TPR_colour_0.0',
       'TPR_colour_0.0-colour_1.0', 'TPR_colour_0.0-colour_2.0',
       'TPR_colour_1.0-colour_2.0', 'TPR_colour_0.0÷colour_1.0',
       'TPR_colour_0.0÷colour_2.0', 'TPR_colour_1.0÷colour_2.0',
       'TNR_colour_2.0', 'TN

In [92]:
bl_cols = {
#     'Clust/Context Accuracy': "Clust Acc",
#     'Clust/Context NMI': "Clust NMI",
#     'Clust/Context ARI': "Clust ARI",
    "Accuracy": "Accuracy",
#     'prob_pos_colour_0.0÷colour_1.0': "AR ratio",
    "Renyi preds and s": "Renyi corr",
#     "TPR_colour_0.0÷colour_1.0": "TPR ratio",
#     'TNR_colour_0.0÷colour_1.0': "TNR ratio",
    "prob_pos ratio mean": "AR, mean ratio",
    "TPR ratio mean": "TPR, mean ratio",
    "TNR ratio mean": "TNR, mean ratio",
    "prob_pos ratio min": "AR, minimum ratio",
    "prob_pos diff max": "AR, maximum diff",
}

In [93]:
groupby = "log_method"
res2 = (baselines[[groupby] + list(bl_cols)]
        .groupby([groupby])
        .agg(Cell(round_to=3)).rename(columns=bl_cols, inplace=False)
        .reset_index(level=[groupby], inplace=False)
        .rename(columns={groupby: "Method"}, inplace=False)
       )
res2

,Method,Accuracy,Renyi corr,"AR, mean ratio","TPR, mean ratio","TNR, mean ratio","AR, minimum ratio","AR, maximum diff"
0,\texttt{FWD \cite{HasSriNamLia18}} (0.01),0.58 $\pm$ 0.035,0.843 $\pm$ 0.034,0.125 $\pm$ 0.033,0.347 $\pm$ 0.04,0.37 $\pm$ 0.058,0.009 $\pm$ 0.023,0.954 $\pm$ 0.071
1,\texttt{FWD \cite{HasSriNamLia18}} (0.1),0.61 $\pm$ 0.059,0.8 $\pm$ 0.072,0.15 $\pm$ 0.066,0.367 $\pm$ 0.09,0.434 $\pm$ 0.11,0.022 $\pm$ 0.049,0.878 $\pm$ 0.123
2,\texttt{FWD \cite{HasSriNamLia18}} (0.3),0.619 $\pm$ 0.044,0.803 $\pm$ 0.047,0.157 $\pm$ 0.066,0.383 $\pm$ 0.097,0.418 $\pm$ 0.098,0.027 $\pm$ 0.05,0.887 $\pm$ 0.101
3,\texttt{FWD \cite{HasSriNamLia18}} (1.0),0.578 $\pm$ 0.043,0.831 $\pm$ 0.051,0.143 $\pm$ 0.08,0.353 $\pm$ 0.111,0.414 $\pm$ 0.121,0.016 $\pm$ 0.059,0.9 $\pm$ 0.127
4,\texttt{Kamiran \& Calders (2012) CNN},0.627 $\pm$ 0.054,0.784 $\pm$ 0.069,0.208 $\pm$ 0.1,0.471 $\pm$ 0.145,0.359 $\pm$ 0.036,0.072 $\pm$ 0.077,0.904 $\pm$ 0.087
5,\texttt{Kamiran \& Calders (2012) CNN} (more s...,0.563 $\pm$ 0.017,0.857 $\pm$ 0.011,0.113 $\pm$ 0.007,0.333 $\pm$ 0.003,0.35 $\pm$ 0.035,0.0009 $\pm$ 0.0016,0.981 $\pm$ 0.035


In [90]:
table = pd.concat([res, res2]).reset_index(drop=True)
table

,Method,Accuracy,Renyi corr,AR ratio mean,TPR ratio mean,TNR ratio mean,AR ratio min,AR diff max
0,\texttt{ZSF},0.882 $\pm$ 0.076,0.264 $\pm$ 0.169,0.725 $\pm$ 0.15,0.911 $\pm$ 0.118,0.798 $\pm$ 0.201,0.604 $\pm$ 0.213,0.236 $\pm$ 0.189
1,\texttt{FWD \cite{HasSriNamLia18}} (0.01),0.58 $\pm$ 0.035,0.843 $\pm$ 0.034,0.125 $\pm$ 0.033,0.347 $\pm$ 0.04,0.37 $\pm$ 0.058,0.009 $\pm$ 0.023,0.954 $\pm$ 0.071
2,\texttt{FWD \cite{HasSriNamLia18}} (0.1),0.61 $\pm$ 0.059,0.8 $\pm$ 0.072,0.15 $\pm$ 0.066,0.367 $\pm$ 0.09,0.434 $\pm$ 0.11,0.022 $\pm$ 0.049,0.878 $\pm$ 0.123
3,\texttt{FWD \cite{HasSriNamLia18}} (0.3),0.619 $\pm$ 0.044,0.803 $\pm$ 0.047,0.157 $\pm$ 0.066,0.383 $\pm$ 0.097,0.418 $\pm$ 0.098,0.027 $\pm$ 0.05,0.887 $\pm$ 0.101
4,\texttt{FWD \cite{HasSriNamLia18}} (1.0),0.578 $\pm$ 0.043,0.831 $\pm$ 0.051,0.143 $\pm$ 0.08,0.353 $\pm$ 0.111,0.414 $\pm$ 0.121,0.016 $\pm$ 0.059,0.9 $\pm$ 0.127
5,\texttt{Kamiran \& Calders (2012) CNN},0.627 $\pm$ 0.054,0.784 $\pm$ 0.069,0.208 $\pm$ 0.1,0.471 $\pm$ 0.145,0.359 $\pm$ 0.036,0.072 $\pm$ 0.077,0.904 $\pm$ 0.087
6,\texttt{Kamiran \& Calders (2012) CNN} (more s...,0.563 $\pm$ 0.017,0.857 $\pm$ 0.011,0.113 $\pm$ 0.007,0.333 $\pm$ 0.003,0.35 $\pm$ 0.035,0.0009 $\pm$ 0.0016,0.981 $\pm$ 0.035


In [91]:
print(table.to_latex(index=False, escape=False))

\begin{tabular}{llllllll}
\toprule
                                            Method &           Accuracy &         Renyi corr &      AR ratio mean &     TPR ratio mean &     TNR ratio mean &         AR ratio min &        AR diff max \\
\midrule
                                      \texttt{ZSF} &  0.882 $\pm$ 0.076 &  0.264 $\pm$ 0.169 &   0.725 $\pm$ 0.15 &  0.911 $\pm$ 0.118 &  0.798 $\pm$ 0.201 &    0.604 $\pm$ 0.213 &  0.236 $\pm$ 0.189 \\
         \texttt{FWD \cite{HasSriNamLia18}} (0.01) &   0.58 $\pm$ 0.035 &  0.843 $\pm$ 0.034 &  0.125 $\pm$ 0.033 &   0.347 $\pm$ 0.04 &   0.37 $\pm$ 0.058 &    0.009 $\pm$ 0.023 &  0.954 $\pm$ 0.071 \\
          \texttt{FWD \cite{HasSriNamLia18}} (0.1) &   0.61 $\pm$ 0.059 &    0.8 $\pm$ 0.072 &   0.15 $\pm$ 0.066 &   0.367 $\pm$ 0.09 &   0.434 $\pm$ 0.11 &    0.022 $\pm$ 0.049 &  0.878 $\pm$ 0.123 \\
          \texttt{FWD \cite{HasSriNamLia18}} (0.3) &  0.619 $\pm$ 0.044 &  0.803 $\pm$ 0.047 &  0.157 $\pm$ 0.066 &  0.383 $\pm$ 0.097 &  0.418 